In [1]:
import tensorflow as tf
import numpy as np
from math import sqrt
from random import randint, shuffle
from tensorflow.keras.layers import Dense, Flatten
import copy

In [2]:
model = tf.keras.Sequential()

model.add(Flatten(input_shape=(9,9)))
model.add(Dense(81*2, activation=tf.math.sigmoid))
model.add(Dense(81*4, activation=tf.math.sigmoid))
model.add(Dense(27*8, activation=tf.math.sigmoid))
model.add(Dense(27*16, activation=tf.math.sigmoid))
model.add(Dense(27*16, activation=tf.math.sigmoid))
model.add(Dense(27*8, activation=tf.math.sigmoid))
model.add(Dense(27*4, activation=tf.math.sigmoid))
model.add(Dense(27*2, activation=tf.math.sigmoid))
model.add(Dense(27*2, activation=tf.math.sigmoid))
model.add(Dense(9+9+9, activation=tf.nn.relu))

print(model.summary())

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError(), metrics=[])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 81)                0         
                                                                 
 dense (Dense)               (None, 162)               13284     
                                                                 
 dense_1 (Dense)             (None, 324)               52812     
                                                                 
 dense_2 (Dense)             (None, 216)               70200     
                                                                 
 dense_3 (Dense)             (None, 432)               93744     
                                                                 
 dense_4 (Dense)             (None, 432)               187056    
                                                                 
 dense_5 (Dense)             (None, 216)               9

2022-03-25 08:52:03.148110: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:

def init(n):
    return [[0 for j in range(n)] for i in range(n)]


def affichage_grille(grille):
    n = len(grille)
    r = int(sqrt(n))
    for i in range(n):
        for j in range(n):
            print(grille[i][j], end="")
            if (j%r==r-1):
                print(" ", end="")
        print()
        if (i%r==r-1):
            print()


def checkGrid(grille):
    n = len(grille)
    for i in range(n):
        for j in range(n):
            if grille[i][j]==0:
                return False
    return True


listeNombres = [1, 2, 3, 4, 5, 6, 7, 8, 9]


def fillGrid(grille):
    n = len(grille)
    r = int(sqrt(n))
    #Find next empty cell
    for i in range(0,n**2):
        row = i//n
        col = i%n
        if grille[row][col] == 0:
            shuffle(listeNombres)
            for value in listeNombres:
                #Check that this value has not already be used on this row
                if not(value in grille[row]):
                #Check that this value has not already be used on this column
                    if not value in [grille[k][col] for k in range(n)]:
                        #Identify which of the 9 squares we are working on
                        isquare = (row//r)*r
                        jsquare = (col//r)*r
                        square=[grille[k][jsquare : jsquare+r] for k in range(isquare, isquare+r)]
                        #Check that this value has not already be used on this 3x3 square
                        if not value in [el for ligne in square for el in ligne]:
                            grille[row][col] = value
                            if checkGrid(grille):
                                return True
                            else:
                                if fillGrid(grille):
                                    return True
            break
    grille[row][col] = 0


def deconstruit_un_coup(grille, movePossibles):
    n = len(grille)
    k = randint(0, len(movePossibles)-1)
    i = movePossibles[k]//n
    j = movePossibles[k]%n
    movePossibles.pop(k)
    move = grille[i][j]
    grille[i][j] = 0
    resultat = [0 for i in range(3*n)]
    resultat[i] = 1
    resultat[n+j] = 1
    resultat[2*n+move-1] = 1
    return resultat


def joue(grille, i, j, move):
    grille[i][j] = move

def complete_grille():
    complet = [
        [1, 2, 3, 4, 5, 6, 7, 8, 9],
        [4, 5, 6, 7, 8, 9, 1, 2, 3],
        [7, 8, 9, 1, 2, 3, 4, 5, 6],
        [2, 1, 4, 3, 6, 5, 8, 9, 7],
        [3, 6, 5, 8, 9, 7, 2, 1, 4],
        [8, 9, 7, 2, 1, 4, 3, 6, 5],
        [5, 3, 1, 6, 4, 2, 9, 7, 8],
        [6, 4, 2, 9, 7, 8, 5, 3, 1],
        [9, 7, 8, 5, 3, 1, 6, 4, 2]
    ]
    for _ in range(100):
        ligne = bool(randint(0,1))
        i = randint(0, 8)
        j = (i//3 * 3) + randint(0, 2)
        if ligne:
            complet[i], complet[j] = complet[j], complet[i]
        else:
            for k in range(9):
                complet[k][i], complet[k][j] = complet[k][j], complet[k][i]
    return complet

def random(grille):
    for _ in range(100):
        ligne = bool(randint(0,1))
        i = randint(0, 8)
        j = (i//3 * 3) + randint(0, 2)
        if ligne:
            grille[i], grille[j] = grille[j], grille[i]
        else:
            for k in range(9):
                grille[k][i], grille[k][j] = grille[k][j], grille[k][i]

def creation_donnees(repetition, profondeurPartie, n):
    train_grilles, train_move = [], []
    for i in range(repetition):
        partie = complete_grille()
        movePossibles = [i*n+j for j in range(n) for i in range(n)]
        for j in range(min(profondeurPartie, n**2)):
            move = deconstruit_un_coup(partie, movePossibles)
            train_grilles.append(copy.deepcopy(partie))
            train_move.append(move)
    return train_grilles, train_move


def melange_jeu(grille, profondeur, movePossibles):
    n = len(grille)
    for i in range(min(profondeur, n**2)):
        deconstruit_un_coup(grille, movePossibles)


def train():
    #Entrainement
    for i in range(1):
        print("n°",i)
        inData, outData = creation_donnees(100, 40, 9)
        model.fit(inData, outData, epochs=20)


def simulation(model, melange):
    partie = complete_grille()
    movePossibles = [i*9+j for j in range(9) for i in range(9)]
    melange_jeu(partie, melange, movePossibles)
    affichage_grille(partie)
    print()
    while (not checkGrid(partie)):
        prediction = model.predict([partie])
        i = max(range(len(prediction[0][0:9])), key=prediction[0][0:9].__getitem__)
        j = max(range(len(prediction[0][9:18])), key=prediction[0][9:18].__getitem__)
        move = 1 + max(range(len(prediction[0][18:27])), key=prediction[0][18:27].__getitem__)
        joue(partie, i, j, move)
        print(i, j, move)


In [11]:
train()

n° 0


KeyboardInterrupt: 

In [9]:
simulation(model, 40)

930 000 610 
300 080 900 
450 002 731 

570 000 006 
005 700 190 
016 040 250 

740 160 009 
129 470 003 
000 824 370 


8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2
8 8 2


KeyboardInterrupt: 

In [20]:
affichage_grille(complete_grille())

for _ in range(10000):
    test = complete_grille()
    movePossibles = [i*9+j for j in range(9) for i in range(9)]
    for _ in range(40):
        deconstruit_un_coup(test, movePossibles)


213 456 789 
879 123 456 
546 789 123 

635 897 214 
987 214 365 
124 365 897 

462 978 531 
798 531 642 
351 642 978 

